In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
import re
import pickle

In [38]:
def scrape_fa(year):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    my_headers = {
        'authority': 'en.wikipedia.org',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
        'cookie': 'WMF-Last-Access=09-Dec-2021; WMF-Last-Access-Global=09-Dec-2021; GeoIP=US:::37.75:-97.82:v4; enwikimwuser-sessionId=c57e79277dd10e90044c; enwikiel-sessionId=361cd58612199630ad70; enwikiwmE-sessionTickLastTickTime=1639063998485; enwikiwmE-sessionTickTickCount=10',
    }
    
    url = 'https://en.wikipedia.org/wiki/List_of_{0}%E2%80%93{1}_NBA_season_transactions'.format(year, str(year+1)[2:])

    response = session.get(url, headers=my_headers)
    print(response)
    soup = BeautifulSoup(response.text)
    
    if year == 2010:
        signed = soup.find_all('table', {'class': 'wikitable'})[5]
        signed_players = list(pd.read_html(str(signed))[0]['Signed in the off-season'].values)
        
        waived = soup.find_all('table', {'class': 'wikitable'})[7]
        waived_players = list(pd.read_html(str(waived))[0]['Player'].values)
    elif year == 2009:
        signed = soup.find_all('table', {'class': 'wikitable'})[5]
        signed_players = signed.find_a
        
        waived = soup.find_all('table', {'class': 'wikitable'})[7]
        waived_players = list(pd.read_html(str(waived))[0]['Player'].values)
    else:
        try:
            for t in soup.find_all('table', {'class': 'wikitable sortable'}):
                ths = str(t.find_all('th'))
                if ('Date signed' in ths) and ('New team' in ths) and ('Former team' in ths):
                    signed_players = list(pd.read_html(str(t))[0]['Player'].values)
                if ('Waived' in ths) or ('waived' in ths):
                    waived_players = list(pd.read_html(str(t))[0]['Player'].values)
        except KeyError:
            print(url)
            return None
    try:
        players = [signed_player for signed_player in signed_players if signed_player not in waived_players]
        df = pd.DataFrame({'Player': players})
        df['previous season'] = str(year-1) + "-" + str(year)[2:]
    except (UnboundLocalError, AttributeError, NameError) as err:
        print(url)
        print(err)
        return None

    return df

In [40]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

response = session.get('https://en.wikipedia.org/wiki/List_of_2009%E2%80%9310_NBA_season_transactions#Free_agency')
soup = BeautifulSoup(response.text)
for t in soup.find_all('table', {'class': 'wikitable sortable'}):
    ths = str(t.find_all('th'))
    if ('Waived' in ths) or ('waived' in ths):
        this_t = t

In [44]:
pd.read_html(str(soup.find_all('table', {'class': 'wikitable'})[5]))[0]

,Signed in the off-season,Signed in the off-season.1,Signed in the off-season.2,Signed in the off-season.3
0,Player,Signed,New team,Former team
1,July,July,July,July
2,Trevor Ariza,July 8,Houston Rockets,Los Angeles Lakers
3,Ron Artest,July 8,Los Angeles Lakers,Houston Rockets
4,Rasheed Wallace,July 8,Boston Celtics,Detroit Pistons
...,...,...,...,...
118,February,February,February,February
119,Mario West,February 1,Atlanta Hawks,10-day Contracts from Atlanta
120,Anthony Tolliver,February 6,Golden State Warriors,10-day Contracts from Golden State
121,March,March,March,March


In [53]:
for t in soup.find_all('table', {'class': 'wikitable sortable'}):
    ths = str(t.find_all('th'))
    if ('Waived' in ths) or ('waived' in ths):
        this_t = t

In [4]:
this_t

NameError: name 'this_t' is not defined

In [51]:
list(pd.read_html(str(this_t))[0]['Player'].values)

['Quinn Cook',
 'Wayne Ellington',
 'Taj Gibson',
 'Ersan İlyasova',
 'Elfrid Payton',
 'Kenny Wooten†',
 'Ky Bowman',
 'Chimezie Metu',
 'Norvel Pelle',
 'Marial Shayok',
 'Khyri Thomas',
 'Keita Bates-Diop',
 'Elie Okobo',
 'Dewayne Dedmon',
 'Jordan Bell',
 'Justin Patton',
 'Dewan Hernandez',
 'Rayjon Tucker',
 'Nicolas Batum',
 'Ray Spalding†',
 'Zhaire Smith',
 'Paul Eboua†',
 'Frank Kaminsky',
 'Omari Spellman',
 'Bruno Caboclo',
 'Yogi Ferrell',
 'Thon Maker',
 'Anžejs Pasečņiks',
 'Dakota Mathias†',
 'Alex Len',
 'Chris Clemons',
 'Jordan Bell',
 'Jordan Bone†',
 'Brodric Thomas†',
 'Ashton Hagans†',
 'Frank Mason III†',
 'Norvel Pelle',
 'Ray Spalding†',
 'DeMarcus Cousins',
 'Damian Jones',
 'André Roberson',
 'Iman Shumpert',
 'Sindarius Thornwell',
 'Noah Vonleh',
 'Marques Bolden†',
 'Quinn Cook',
 'Shaquille Harrison',
 'Glenn Robinson III',
 'Jaylen Adams†',
 'Blake Griffin',
 'Mason Jones†',
 'LaMarcus Aldridge',
 'Mfiondu Kabengele',
 'Jalen Lecque',
 'Meyers Leonard'

In [39]:
with Pool(12) as p:
    free_agents = p.map(scrape_fa, list(range(2005, 2021)))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]><Response [200]>

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


KeyError: 'Player'

In [27]:
free_agents = pd.concat(free_agents, axis=0)

In [28]:
free_agents

,Player,previous season
0,Earl Watson,2004-05
1,Raja Bell,2004-05
2,José Calderón,2004-05
3,Samuel Dalembert,2004-05
4,Antonio Daniels,2004-05
...,...,...
403,Lance Thomas,2019-20
404,Noah Vonleh,2019-20
405,Dion Waiters,2019-20
406,Greg Whittington,2019-20


In [16]:
with open('free_agents.pickle', 'wb') as handle:
    pickle.dump(free_agents, handle, protocol=pickle.HIGHEST_PROTOCOL)